In [14]:
import numpy as np
import supervision as sv
import cv2
from ultralytics import YOLO

In [15]:
model = YOLO(r"C:\Users\dolap\OneDrive\Documents\DOLAPO\data-analysis\fish-behavior-analysis\model\best.pt")

In [16]:
tracker = sv.ByteTrack()
mask_annotator = sv.MaskAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

In [17]:
def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {model.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = mask_annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

In [ ]:
sv.process_video(
    source_path= r"C:\Users\dolap\OneDrive\Documents\DOLAPO\data-analysis\fish-behavior-analysis\data\test_video.mp4",
    target_path="result.mp4",
    callback=callback
)

print("Processing complete. The result is saved as 'result.mp4'.")